# Activity 3.1: Classifying fashion clothes using TensorFlow Dataset and TensorFlow 2. 

1. Import all required modules

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib.pyplot as plt

# TensorFlow and Keras
import tensorflow as tf
import tensorflow_datasets as tfds

2. Import the Fashion MNIST dataset using TensorFlow Datasets splitting it into train and test splits and create the list of classes 

In [ ]:
# Construct a tf.data.Dataset
(train_images, train_labels), (test_images, test_labels) = tfds.as_numpy(tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    batch_size=-1,
    as_supervised=True,
))

train_images = np.squeeze(train_images)
test_images = np.squeeze(test_images)

classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

3. Explore the dataset to get familiar with input features shapes, labels, classes, their visual appearance and perform data normalization 

In [ ]:
print("Training dataset shape =", train_images.shape)
print("Training labels length =", len(train_labels))
print("Some training labels =", train_labels[:5])
print("Test dataset shape =", test_images.shape)
print("Test labels length =", len(test_labels))

It is also useful to take a look at how images appear. The following code snippet shows the first training set instance

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Perform features normalization 

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

Let’s now take a look at some instances of our training set, by plotting 25 of them with their correspondent label: 

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(classes[train_labels[i]])
plt.show()

4. Build the classification model

First we create a model creating a layers sequence: 

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

Then we associate to the model an optimizer, a loss function and a metric: 

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

5. Train the deep neural network

In [ ]:
model.fit(train_images, train_labels, epochs=10)

6. Test model accuracy. You should obtain accuracy above 88%. 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy:', test_accuracy)

7. Perform inference and check predictions against ground truth 

As a first step we add a softmax layer to the model, so that it outputs probabilities instead of logits, and print out the probabilities of the first 3 test instances with the following code: 

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(test_images)
print(predictions[0:3])

Next we compare one model prediction (i.e. the class with the highest predicted probability), the one on the first test instance, with its ground truth

In [ ]:
print("Class ID, predicted | real =", np.argmax(predictions[0]), "|", test_labels[0])

In order to perform a comparison even more clear, we create the following two functions. The first one plots the i-th test set instance image with a caption showing the predicted class with the highest probability, its probability in percentage, and the ground truth between round brackets. This caption will be blue for correct predictions, and red for wrong ones. 

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(classes[predicted_label],
                                  100*np.max(predictions_array),
                                  classes[true_label]),
                                  color=color)

The second function creates a second image showing a bar plot of all classes predicted probabilities. Coloring the highest probable one in blue if the prediction is correct, or in red if it is incorrect. In this second case the bar corresponding to the correct label is colored in blue. 

In [ ]:
def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('blue')

Using these two functions we can examine every instance of the test set. In the following snippet we plot the first test instance.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

The very same approach can be used to plot a user defined numer of test instances, arranging the output in sub plots as follows: 

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions[i], test_labels, test_images)
    plt.subplot(num_rows, 2*num_cols, 2*i+2)
    plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()